In [8]:
import torch
from torch import nn
from torch import optim

import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [9]:
class CustomDataset(Dataset):

    def __init__(self, dataframe: pd.DataFrame, feature_names: list, target_name: str):
        X = torch.tensor(dataframe[feature_names].values, dtype=torch.float32)
        self.X = X.reshape(X.shape[0], 1, X.shape[1])
        y = torch.tensor(dataframe[target_name].values, dtype=torch.float32)
        self.y = y.type(torch.LongTensor)
        
    def __len__(self) -> int:
        return len(self.y)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [10]:
class FullyConnectedNet(nn.Module):

    def __init__(self, in_size: int, hidden_size: list, out_size: int):
        super(FullyConnectedNet, self).__init__()

        self.Network = nn.Sequential(
            nn.Linear(in_size, hidden_size[0]),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2, inplace=True),
            nn.Linear(hidden_size[0], hidden_size[1]),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2, inplace=True),
            nn.Linear(hidden_size[1], hidden_size[2]),
            nn.ReLU(),
            nn.Dropout(0.2, inplace=True),
            nn.Linear(hidden_size[2], out_size),
            nn.Softmax()
        )
    
    def forward(self, x):
        pass



In [11]:
df = pd.read_csv("data\df_01.csv", index_col=[0])
df = df.reset_index().drop(columns=['index'])

In [12]:
net = FullyConnectedNet(in_size=18, hidden_size=[144, 72, 36], out_size=12)
print(net)

FullyConnectedNet(
  (Network): Sequential(
    (0): Linear(in_features=18, out_features=144, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.2, inplace=True)
    (3): Linear(in_features=144, out_features=72, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.2, inplace=True)
    (6): Linear(in_features=72, out_features=36, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.2, inplace=True)
    (9): Linear(in_features=36, out_features=12, bias=True)
    (10): Softmax(dim=None)
  )
)


In [13]:
optimizer = optim.Adam(params=net.parameters(), lr=0.001, weight_decay=0.001)
loss_function = nn.CrossEntropyLoss()